In [45]:
## Generate counter_tb.sv using a yield-based python model of the counter
import numpy as np

def get_stimulus():
    yield True, False
    for _ in range(10):
        yield False, False
    for _ in range(20):
        yield False, True
    for _ in range(10):
        yield False, False

def counter_model(stimulus, **kwargs):
    increment = kwargs['increment'] if 'increment' in kwargs else 1
    reset_value = kwargs['reset_value'] if 'reset_value' in kwargs else 0
    width = kwargs['width'] if 'width' in kwargs else 4
    max_value = kwargs['max_value'] if 'max_value' in kwargs else 4
    mask = ((1 << width) - 1)
    clock = 0
    count = reset_value
    for reset, enable in stimulus:
        carry = (count + increment >= max_value)
        yield clock, reset, enable, count, carry
        if reset:
            count = reset_value & mask
        elif enable:
            if carry:
                count = reset_value & mask
            else:
                count = (count + increment) & mask
        clock += 1

def write_golden(**kwargs):
    width = kwargs['width'] if 'width' in kwargs else 4
    reset_value = kwargs['reset_value'] if 'reset_value' in kwargs else 0
    increment = kwargs['increment'] if 'increment' in kwargs else 1
    max_value = kwargs['max_value'] if 'max_value' in kwargs else 9
    with open('hdl/counter_tb.sv', 'w') as golden:
        golden.writelines(f"module counter_tb;\n")
        golden.writelines(f"    logic clk = 0;\n")
        golden.writelines(f"    always #5 clk = ~clk;\n")
        golden.writelines(f"    logic reset;\n")
        golden.writelines(f"    logic enable;\n")
        golden.writelines(f"    logic [{width-1}:0] count;\n")
        golden.writelines(f"    logic carry;\n")
        golden.writelines(f"    counter #(\n"
                          f"        .width({width}),\n"
                          f"        .reset_value({reset_value}),\n"
                          f"        .increment({increment}),\n"
                          f"        .max_value({max_value})\n"
                          f"    ) dut (.*);\n")
        result = counter_model(get_stimulus(), **kwargs)
        _, reset, enable, count, carry = result.__next__()
        golden.writelines(f"    initial begin\n")
        golden.writelines(f"        reset = {1 if reset else 0};\n")
        golden.writelines(f"        enable = {1 if enable else 0};\n")
        last_reset = None
        last_enable = None
        for _, reset, enable, count, carry in result:
            golden.writelines(f"        @(posedge clk);\n")
            if last_reset != reset:
                golden.writelines(f"        reset <= {1 if reset else 0};\n")
            if last_enable != enable:
                golden.writelines(f"        enable <= {1 if enable else 0};\n")
            last_reset, last_enable = reset, enable
            # golden.writelines(f"        assert (count == 'd{count} && carry == {1 if carry else 0}) else\n")
            # golden.writelines(f"            $fatal(1, \"Something didn't match the expected result\");\n")
        golden.writelines(f"        @(posedge clk);\n")
        golden.writelines(f"        $display(\"Test passed\");\n")
        golden.writelines(f"        $finish;\n")
        golden.writelines(f"    end\n")
        golden.writelines(f"endmodule\n")

write_golden(width=4, reset_value=1, increment=3, max_value=15)
